In [1]:
import skimage
import os
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib
import matplotlib.pyplot as plt
from skimage import data
from skimage import io
from sklearn.mixture import GaussianMixture
import glob
import keras
import tqdm
from skimage.viewer import ImageViewer
import cv2
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import stop_words
from keras.preprocessing import image
from keras.models import Sequential
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, Activation
from keras.preprocessing.image import ImageDataGenerator
from keras import utils
%matplotlib inline

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/anaconda3/lib/python3.6/site-packages/skimage/viewer/utils/core.py:10: UserWarning: Recommended matplotlib backend is `Agg` for full skimage.viewer functionality.
  warn("Recommended matplotlib backend is `Agg` for full "


In [2]:
#Define File-Path to Users Folder
folder_list = ['#foodporn',"#nightlife","#cosmetics","#rockclimbing"]
image_path = "/Users/kmotwani/Desktop/Me/Education/Courses/Capstone Project/Google Images/"

In [3]:
#Helper Function to get images from path
def get_df(path, list_input, thresh):
    final_list = []
    for ind, i in enumerate(list_input):
        temp_path = path + i
        print(i,"\n")
        count = 0
        for j in glob.glob(temp_path + '/*.jpg'):
            temp_dict = {}
            file_name = j.replace(temp_path,'')[1:]
            img = image.load_img(j, target_size=(128, 128))
            try:
                with open(temp_path+"/"+file_name[:-4]+'.txt', encoding="utf-8") as f:
                    content = f.readlines()
                    caption = ' '.join([x.strip() for x in content])
            except FileNotFoundError:
                continue
            temp_dict['File'], temp_dict['Response'] = file_name, int(ind)
            temp_dict['Image'], temp_dict['Caption'] = np.array(img), caption
            final_list.append(temp_dict)
            count += 1
            if count==thresh:
                break
    return pd.DataFrame(final_list) 


#Get Images from User List and Path
df = get_df(image_path, folder_list, 1000)
print("Number of images loaded:", len(df))
display(df.head())

#foodporn 

#nightlife 

#cosmetics 

#rockclimbing 

Number of images loaded: 4000


,Caption,File,Image,Response
0,...Siempre quise ser grande para poder comerme...,2018-04-08_21-17-49_UTC.jpg,"[[[82, 68, 59], [88, 85, 96], [128, 104, 80], ...",0
1,Lonche de Panela By @mojito.restobar . . . #va...,2018-04-08_21-36-39_UTC.jpg,"[[[165, 158, 199], [162, 155, 196], [167, 159,...",0
2,Brunching with friends is the best way to spen...,2018-04-08_20-19-50_UTC.jpg,"[[[61, 51, 49], [48, 37, 31], [153, 149, 166],...",0
3,Obsessed with their little stoves. Ordered the...,2018-02-02_15-27-17_UTC.jpg,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",0
4,"Hoje para o jantar foram dois ovos, em cima de...",2018-04-08_21-36-29_UTC.jpg,"[[[147, 163, 160], [152, 163, 155], [131, 101,...",0


In [ ]:
#Split train and test set
use_df = df.sample(frac=1).reset_index(drop=True)
np.random.seed(9001)
msk = np.random.rand(len(use_df)) < 0.7
total_data_train = use_df[msk]
total_data_test = use_df[~msk]

#Define X,Y
x_train = np.array([i for i in total_data_train['Image'].as_matrix()])
x_test = np.array([i for i in total_data_test['Image'].as_matrix()])
y_train = utils.to_categorical(total_data_train['Response'].as_matrix(), num_classes=4)
y_test = utils.to_categorical(total_data_test['Response'].as_matrix(), num_classes=4)

In [10]:
#Helper function to create CNN Model for Image Classification
def createModel(shape, classes_count):
    model = Sequential()
    model.add(Conv2D(32, (5, 5), input_shape=(128, 128, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(32, (5, 5), activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(classes_count, activation='softmax'))
    model.summary()
    return model

#Helper function to run model and save intermediate weights
def run_model(model, x_train, y_train, x_test, y_test, batch_size, epochs):
    model.compile(loss='categorical_crossentropy',optimizer=keras.optimizers.Adam(lr=1e-4),metrics=['accuracy'])
    filepath="Saved_Models/KRM_weights-{epoch:02d}-{val_acc:.2f}.hdf5"
    check = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max', period = 10)
    callbacks_list = [check]
    history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, 
                       validation_data=(x_test, y_test), callbacks=callbacks_list)
    return model, history

In [ ]:
#Create and Run model
model = createModel((128,128,3), 4)
run_model(model, x_train, y_train, x_test, y_test, batch_size=512, epochs=2000)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 124, 124, 32)      2432      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 62, 62, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 58, 58, 32)        25632     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 29, 29, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 25, 25, 64)        51264     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 9216)              0         
__________

2791/2791 [==============================] - 102s 36ms/step - loss: 0.6140 - acc: 0.7324 - val_loss: 1.1753 - val_acc: 0.5864
Epoch 44/2000
2791/2791 [==============================] - 100s 36ms/step - loss: 0.5944 - acc: 0.7510 - val_loss: 1.1967 - val_acc: 0.5914
Epoch 45/2000
2791/2791 [==============================] - 96s 34ms/step - loss: 0.5852 - acc: 0.7506 - val_loss: 1.2023 - val_acc: 0.5964
Epoch 46/2000
2791/2791 [==============================] - 84s 30ms/step - loss: 0.5691 - acc: 0.7635 - val_loss: 1.2133 - val_acc: 0.5848
Epoch 47/2000
2791/2791 [==============================] - 251s 90ms/step - loss: 0.5531 - acc: 0.7718 - val_loss: 1.2328 - val_acc: 0.5873
Epoch 48/2000
2791/2791 [==============================] - 1747s 626ms/step - loss: 0.5314 - acc: 0.7836 - val_loss: 1.2481 - val_acc: 0.5922
Epoch 49/2000
1536/2791 [===============>..............] - ETA: 36s - loss: 0.5464 - acc: 0.7832

In [ ]:
count = 0
for ind,i in enumerate(x_train):
    count+=1
    if count<10:
        plt.figure()
        plt.imshow(i)
        print(y_train[ind])
    else:
        break